In [ ]:
# Importieren der erforderlichen Bibliotheken
import random
from pyspark.sql import functions as F
from pyspark.sql.functions import  col
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import count, avg, desc, round as round_func
import time
from pyspark.sql.functions import when

In [ ]:
# Spark Session initialisieren
spark = SparkSession.builder \
    .appName("MovieLens_Angriffsprofile") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()


# Lesen der Ratings-Datei
ratings_df = spark.read.csv("ml-32m/ratings.csv", header=True, inferSchema=True)
#ratings_df = ratings_df.drop('timestamp')

# Laden der movies.csv
movies_df = spark.read.csv('ml-32m/movies.csv', header=True, inferSchema=True)

# Verbinden der DataFrames basierend auf der movieId
ratings = ratings_df.join(movies_df, on='movieId', how='inner')

# Schema und erste 10 Einträge des Datensatzes
ratings.printSchema()
ratings.show(10)

In [ ]:
# Anzahl der Ratings im Datensatz
print("Anzahl der Ratings im Datensatz:", ratings.count())

# Duplikate filtern
duplicates = (
    ratings.groupBy("userId", "movieId")
    .count()
    .filter(F.col("count") > 1)
)

# Duplikate anzeigen
print("Anzahl der Duplikate:", duplicates.count())

# Duplikate entfernen
ratings = ratings.dropDuplicates(["userId", "movieId"])

# Ergebnis überprüfen: Anzahl der Ratings im Datensatz nach Entfernung der Duplikate
print("Anzahl der Zeilen nach Entfernen der Duplikate:")
print(ratings.count())

In [ ]:
# Berechne den Mittelwert und die Standardabweichung der Ratings im Datensatz und zeige diese an
mean_rating_system = ratings.agg({"rating": "avg"}).collect()[0][0]
stddev_rating = ratings.agg({"rating": "stddev"}).collect()[0][0]

print(mean_rating_system)
print(stddev_rating)

In [ ]:
# Anzahl aller Ratings im DataFrame
total_ratings_counte = ratings_df.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_counte)

In [ ]:
# Berechnung der Beliebtheit und der durchschnittlichen Bewertung pro Film
item_popularity_and_avg_rating = (
    ratings.groupBy("movieId")
    .agg(
        F.count("rating").alias("num_ratings"),
        F.avg("rating").alias("avg_rating")
    )
)

# Finde das mittlere Quantil für die Anzahl der Bewertungen
quantiles = item_popularity_and_avg_rating.approxQuantile("num_ratings", [0.4, 0.6], 0.05)
min_num_ratings, max_num_ratings = quantiles[0], quantiles[1]

# Mittlere Bewertung: Finde das mittlere Quantil für die durchschnittliche Bewertung
rating_quantiles = item_popularity_and_avg_rating.approxQuantile("avg_rating", [0.4, 0.6], 0.05)
min_avg_rating, max_avg_rating = rating_quantiles[0], rating_quantiles[1]

# Wähle repräsentatives Genre
popular_genres = (
    movies_df
    .withColumn("genre", F.explode(F.split(F.col("genres"), "\|")))
    .groupBy("genre")
    .count()
    .orderBy(F.desc("count"))
    .limit(3)  # Nimm die Top 3 Genres als Beispiel
)

# Filtere die Filme mit mittlerer Beliebtheit, mittlerer Bewertung und einem repräsentativen Genre
target_candidates = (
    item_popularity_and_avg_rating
    .filter(
        (F.col("num_ratings") >= min_num_ratings) & (F.col("num_ratings") <= max_num_ratings) &
        (F.col("avg_rating") >= min_avg_rating) & (F.col("avg_rating") <= max_avg_rating)
    )
    .join(movies_df, "movieId")
    .withColumn("genre", F.explode(F.split(F.col("genres"), "\|")))  # Genre spalten
    .join(popular_genres, "genre")  # Nur beliebte Genres
)

# Zeige ein paar Kandidaten, die als Target-Item verwendet weden könnten
target_candidates.show(5)

In [ ]:
# Liste aller verfügbaren MovieIDs aus dem Datensatz abrufen
all_movie_ids = ratings.select("movieId").distinct().rdd.flatMap(lambda x: x).collect()

# Funktion zum Validieren der eingegebenen MovieID
def get_valid_target_item():
    while True:
        user_input = input("Bitte gib die movieId des Ziel-Items ein oder tippe 'zufällig' für ein zufälliges Item: ")
        
        if user_input.lower() == 'zufällig':
            # Wähle ein zufälliges gültiges Item aus der Liste
            return random.choice(all_movie_ids)
        
        try:
            # Nutzeraufforderung in eine Zahl umwandeln
            target_item = int(user_input)
            
            # Überprüfen, ob die eingegebene MovieID im Datensatz vorhanden ist
            if target_item in all_movie_ids:
                return target_item
            else:
                print("Die eingegebene movieId ist ungültig. Bitte versuche es erneut.")
        except ValueError:
            print("Ungültige Eingabe! Bitte gib eine gültige numerische movieId ein oder tippe 'zufällig'.")

# Abrufen der validierten MovieID vom Nutzer
target_item = get_valid_target_item()

print(f"Das Ziel-Item (movieId) für den Angriff lautet: {target_item}")

In [ ]:
# Gesamtanzahl der User Profile im Datensatz
total_user_profiles = ratings.select("userId").distinct().count()

# Aufforderung, einen %-Anteil für die Anzahl der Angriffsprofile einzugeben
percentage_of_attacks = float(input("Bitte gib den Prozentsatz der Angriffsprofile ein (z.B. 5 für 5%): "))

# Berechne die Anzahl der Angriffsprofile basierend auf dem eingegebenen Prozentsatz
num_attack_profiles = int((percentage_of_attacks / 100) * total_user_profiles)

print(f"Die Anzahl der Angriffsprofile beträgt: {num_attack_profiles}")

In [ ]:
# Berechnung der Durchschnittsbewertungen pro MovieID
item_avg_ratings = ratings.groupBy("movieId").agg(avg("rating").alias("avg_rating"))
item_avg_ratings.show(10)
# Konvertiere item_avg_ratings in ein Dictionary für schnellen Zugriff
item_avg_ratings_dict = {row["movieId"]: row["avg_rating"] for row in item_avg_ratings.collect()}

In [ ]:
# Berechne die Top 20 populärsten Filme nach Anzahl der Bewertungen
popular_items = ratings.groupBy("movieId").count().orderBy(F.col("count").desc()).limit(20)

# Filtere das Target-Item heraus, bevor die Liste erstellt wird
popular_items_list = [row['movieId'] for row in popular_items.collect() if row['movieId'] != target_item]

print("Top 20 populäre Items (movieId), ohne das Target-Item:", popular_items_list)

In [ ]:
# DataFrame mit movieId und Genres auswählen
movies_with_genres_df = ratings.select("movieId", "genres")

# Funktion zum Aufteilen der Genres und Erstellen eines Paares (genre, movieId)
def split_genres(row):
    movie_id = row["movieId"]
    genres = row["genres"].split("|")
    return [(genre, movie_id) for genre in genres]

# RDD erstellen, indem die Genres aufteilt werden und dann nach Genre gruppiert werden
movies_by_genre_rdd = movies_with_genres_df.rdd.flatMap(split_genres).groupByKey()

# Konvertiere das RDD in ein Dictionary, um schnellen Zugriff auf movieIds nach Genre zu ermöglichen
movies_by_genre = movies_by_genre_rdd.mapValues(list).collectAsMap()

# Ziel-Item abfragen
target_item_genres = movies_with_genres_df.filter(F.col("movieId") == target_item).select("genres").collect()[0]["genres"].split('|')

# Liste aller Kandidaten für Segment-Items erstellen
segment_movies_candidates = []

for genre in target_item_genres:
    if genre in movies_by_genre:
        for movie in movies_by_genre[genre]:
            # Nur hinzufügen, wenn das Segment-Item nicht das Target-Item ist und nicht bereits vorhanden ist
            if movie != target_item and movie not in segment_movies_candidates:
                segment_movies_candidates.append(movie)

# Zufällige Auswahl von 20 Segment-Items aus den Kandidaten
segment_movies = random.sample(segment_movies_candidates, 20) if len(segment_movies_candidates) > 20 else segment_movies_candidates

print(segment_movies_candidates)

# Ausgabe der ausgewählten Segment-Items
print(segment_movies)

In [ ]:
# Funktion zur Generierung von Filler-Ratings nach Normalverteilung, gerundet auf 0.5-Schritte
def generate_filler_rating(mean_rating):
    # Generiere eine Bewertung nach der Normalverteilung basierend auf dem übergebenen Mittelwert
    rating = np.random.normal(mean_rating, stddev_rating)
    # Begrenze das Rating auf die Skala von 1 bis 5
    rating = max(1.0, min(5.0, rating))
    # Runde das Rating auf die nächste halbe Zahl (1, 1.5, 2, ..., 5)
    return round(rating * 2) / 2

In [ ]:
# Angriffstyp auswählen und validieren
attack_type = input("Wähle Angriffstyp ('bandwagon', 'random', 'average' oder 'segment'): ").lower()
while attack_type not in ['bandwagon', 'random', 'average', 'segment']:
    print("Ungültige Eingabe! Bitte wähle entweder 'bandwagon', 'random', 'average' oder 'segment'.")
    attack_type = input("Wähle Angriffstyp ('bandwagon', 'random', 'average' oder 'segment'): ").lower()

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Gruppiere nach 'userId' und zähle die Ratings
ratings_per_user = ratings.groupBy("userId").agg(count("rating").alias("num_ratings"))

# Berechne den Durchschnitt der Ratings pro Benutzer
average_ratings_per_user = ratings_per_user.agg(avg("num_ratings")).collect()[0][0]

# Runde den Durchschnitt und ziehe 1 ab
rounded_average = round(average_ratings_per_user - 1)

# Ausgabe der durchschnittlichen Anzahl an Ratings pro Profil abzüglich von 1
print("Durchschnittliche Anzahl an Ratings pro Profil (gerundet, nach Abzug von 1):", rounded_average)

In [ ]:
# Funktion zur Generierung einer Profilgröße basierend auf einer Normalverteilung
def generate_attack_profile_size(average):
    # Generiere eine Anzahl von Bewertungen basierend auf einer Normalverteilung
    std_dev = 0.5 * average  # Standardabweichung = 50% des Durchschnitts
    profile_size = int(np.random.normal(average, std_dev))

    # Stelle sicher, dass die Profillänge mindestens 1 ist
    profile_size = max(profile_size, 1)
    
    return profile_size

In [ ]:
# Funktion zum Erstellen von Angriffs-Bewertungen für jeden neuen User
def generate_attack_ratings(user_id, target_item, popular_items_list, all_movie_ids, attack_type, segment_movies):
    ratings_list = []
    
    # Ziel-Item mit der maximalen Bewertung (5) bewerten
    ratings_list.append((user_id, target_item, 5.0, get_current_timestamp()))

    # Bestimme die Anzahl der Filler-Items basierend auf der Normalverteilung
    num_fillers = generate_attack_profile_size(rounded_average)

    # Reduziere die Anzahl der Filler-Items bei Bandwagon und Segment-Angriffen
    if attack_type == 'bandwagon' or attack_type == 'segment':
        num_fillers = max(num_fillers - 20, 0)  # Setze Untergrenze auf 0
    
    # Füge populäre Items hinzu bei Bandwagon
    if attack_type == 'bandwagon':
        # 20 populärste Items mit 5 bewerten
        for movie_id in popular_items_list:
            ratings_list.append((user_id, movie_id, 5.0, get_current_timestamp()))
    
    # Füge Segment-Items hinzu bei Segment-Angriff
    elif attack_type == 'segment' and segment_movies:
        # 20 Items aus der vorbereiteten Segment-Liste bewerten
        for movie_id in segment_movies:
            ratings_list.append((user_id, movie_id, 5.0, get_current_timestamp()))

    # Liste der nicht erlaubten Items (immer das Target-Item)
    not_allowed_items = set([target_item])  # Set für Effizienz

    # Füge bei Bandwagon-Angriff die populären Items hinzu
    if attack_type == 'bandwagon':
        not_allowed_items.update(popular_items_list)
    
    # Füge bei Segment-Angriff die Segment-Items hinzu
    if attack_type == 'segment' and segment_movies:
        not_allowed_items.update(segment_movies)
    
    # Filtere die Filler-Items, sodass sie keine "nicht erlaubten" Items enthalten
    available_filler_items = [movie_id for movie_id in all_movie_ids if movie_id not in not_allowed_items]

    # Falls num_fillers größer ist als die verfügbare Anzahl an Filler-Items, passe es an
    num_fillers = min(num_fillers, len(available_filler_items))
    
    # Wähle zufällige Filler-Items aus den verbleibenden Filmen aus
    random_movies = random.sample(available_filler_items, num_fillers)
    
    # Filler-Items bewerten
    for movie_id in random_movies:
        if attack_type == 'average':
            mean_rating = item_avg_ratings_dict.get(movie_id, mean_rating_system)
            filler_rating = generate_filler_rating(mean_rating)
        elif attack_type == 'segment':
            filler_rating = 1.0  # Bei Segment-Angriffen werden Filler-Items immer mit 1 bewertet
        else:
            filler_rating = generate_filler_rating(mean_rating_system)

        ratings_list.append((user_id, movie_id, filler_rating, get_current_timestamp()))
    
    return ratings_list

# Funktion zur Generierung des aktuellen Timestamps mit kleiner Zufallsabweichung
def get_current_timestamp():
    # Hol den aktuellen Unix-Timestamp
    current_timestamp = int(time.time())
    
    # Füge eine zufällige Zeitabweichung zwischen 0 und 300 Sekunden hinzu
    time_offset = random.randint(0, 300)
    
    return current_timestamp + time_offset

# Liste aller verfügbaren Filme
all_movie_ids = ratings.select("movieId").distinct().rdd.flatMap(lambda x: x).collect()

# Generiere Angriffs-Bewertungen und speichere sie als DataFrame
attack_ratings = []
for i in range(num_attack_profiles):
    user_id = 100000000 + i  # Erstelle eine neue UserID für jeden Angriff (angenommen IDs >= 100000000 sind neu)
    
    # Erzeuge Angriffsbewertungen
    attack_ratings.extend(generate_attack_ratings(
        user_id, 
        target_item, 
        popular_items_list, 
        all_movie_ids, 
        attack_type, 
        segment_movies
    ))

# Erstelle ein DataFrame mit den generierten Angriffsbewertungen
attack_ratings_df = spark.createDataFrame(attack_ratings, ["userId", "movieId", "rating", "timestamp"])

# Zeige die ersten 10 Einträge des Angriffs-Dataframes
attack_ratings_df.show(10)

In [ ]:
# Anzahl der Einträge im attack_ratings_df
num_entries = attack_ratings_df.count()
print(f"Anzahl der Einträge (Bewertungen) in Angriffsratings: {num_entries}")

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Anzahl verschiedener userIds
num_users = attack_ratings_df.select("userId").distinct().count()
print(f"Anzahl verschiedener User IDs in Angriffsratings: {num_users}")

In [ ]:
# Anzahl verschiedener movieIds
num_movies = attack_ratings_df.select("movieId").distinct().count()
print(f"Anzahl verschiedener Movie IDs in Angriffsratings: {num_movies}")

In [ ]:
# Häufigkeit der Ratings im Angriffs-Dataframe
rating_counts = attack_ratings_df.groupBy("rating").count().orderBy("rating")
rating_counts.show()

# Alternativ als Python-Dictionary
rating_counts_dict = {row['rating']: row['count'] for row in rating_counts.collect()}
print(f"Häufigkeit der Bewertungen in Angriffsratings: {rating_counts_dict}")

In [ ]:
# Nur relevante Spalten aus dem ursprünglichen ratings DataFrame auswählen
ratings = ratings.select("userId", "movieId", "rating", "timestamp")

In [ ]:
# Ursprüngliche Daten und Angriffs-Daten kombinieren
all_ratings = ratings.union(attack_ratings_df)
all_ratings.show(10)

In [ ]:
# Häufigkeit der Ratings im Gesamt-Dataframe
rating_counts2 = all_ratings.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

In [ ]:
# Label-Spalte hinzufügen: 0 für normale User (vorhandene), 1 für Angriffs-User
all_ratings = all_ratings.withColumn("Label", 
                                     when(col("userId") >= 100000000, 1)
                                     .otherwise(0))

# Überprüfen des finalen Datensatzes
all_ratings.show(10)

In [ ]:
# Variable mit Namen des Datensatzes
datensatz = "MovieLens32M"

In [ ]:
# Dynamischer Dateiname basierend auf Datensatz, Angriffsart und Angriffsgröße
dateiname = f"all_ratings_{datensatz}_{attack_type}_{percentage_of_attacks}.csv"

# Speichern des DataFrames mit dem dynamischen Dateinamen
all_ratings.coalesce(1).write.csv(dateiname, header=True)

In [ ]:
# Filter auf userId >= 100000000 (Angreifer)
filtered_ratings = all_ratings.filter(col("userId") >= 100000000)

# Anzeigen der gefilterten Zeilen
filtered_ratings.show(10)

In [ ]:
# Filter auf userId >= 100000000
filtered_ratings2 = all_ratings.filter(col("userId") == 100000000)

# Anzeigen der Bewertungen von Nutzer mit userId 100000000
filtered_ratings2.show(10)

# Häufigkeit der Ratings bei User 100000000
rating_counts2 = filtered_ratings2.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

# Alternativ als Python-Dictionary
rating_counts_dict2 = {row['rating']: row['count'] for row in rating_counts2.collect()}
print(f"Häufigkeit der Bewertungen: {rating_counts_dict2}")

In [ ]:
# Berechne den Durchschnitt der Bewertungen für userId 100000000
average_rating_user = attack_ratings_df.filter(col("userId") == 100000000).agg(avg("rating")).collect()[0][0]

print(f"Der Durchschnitt der Bewertungen von User 100000000 ist: {average_rating_user}")

In [ ]:
# Berechne die neue durchschnittliche Bewertung im Gesamt-Dataframe nach einem Angriff
mean_rating2 = all_ratings.agg({"rating": "avg"}).collect()[0][0]
print(mean_rating2)

In [ ]:
# Anzahl aller Ratings im Angriffs-DataFrame
total_ratings_count_attack = attack_ratings_df.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Angriffsratings:", total_ratings_count_attack)

In [ ]:
# Anzahl aller Ratings im Gesamt-Dataframe
total_ratings_count = all_ratings.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_count)

In [ ]:
# Definiere die User-IDs für die Abfrage
user_ids_mio = [i for i in range(100000000, 100000021)]  # User von 100000000 bis 100000020

# Filtere den Datensatz für die gewünschten User und zähle die Ratings pro User
ratings_count_mio = all_ratings.filter(F.col("userId").isin(user_ids_mio)) \
                       .groupBy("userId") \
                       .agg(F.count("rating").alias("num_ratings")) \
                       .orderBy("userId")

# Zeige die Anzahl der Ratings für die angegebenen User
ratings_count_mio.show()

In [ ]:
# Beende die Spark-Session
spark.stop()